## GPT-2 Fine-tuning на датасете Wikibooks: Пошаговое руководство

В этом туториале мы погрузимся в процесс **классического дообучения (full fine-tuning)** предобученной языковой модели GPT-2 на специфическом корпусе текстов — датасете Wikibooks на русском языке. Full fine-tuning — это мощный метод в области машинного обучения, который позволяет адаптировать уже обученную модель к новой задаче или набору данных, обновляя при этом **все её параметры**.

### Что такое Full Fine-tuning?

Full fine-tuning — это процесс взятия предварительно обученной нейронной сети (в нашем случае, GPT-2) и дальнейшего обучения её на новом, часто меньшем и более специализированном наборе данных. Идея заключается в том, что большая языковая модель, такая как GPT-2, уже изучила обширные общие закономерности языка (грамматику, синтаксис, базовые факты) на огромных объемах текстовых данных. Однако для выполнения более конкретных задач или работы с уникальными стилями текста ей требуется дополнительная адаптация.

В отличие от методов PEFT (Parameter-Efficient Fine-Tuning), которые обновляют лишь малую часть параметров модели или добавляют небольшие, обучаемые модули, **full fine-tuning обновляет абсолютно все веса и смещения в нейронной сети**.

### Как это работает?

Процесс full fine-tuning можно упрощенно описать так:

1.  **Выбор предобученной модели:** мы начинаем с модели, которая уже прошла обучение на очень большом и разнообразном корпусе данных. В нашем случае это `ai-forever/rugpt3small_based_on_gpt2` — русскоязычная версия GPT-2.
2.  **Подготовка нового датасета:** собирается меньший, целевой набор данных, который специфичен для нашей задачи. В этом туториале мы используем часть датасета Wikibooks, чтобы модель научилась генерировать тексты в стиле энциклопедических статей.
3.  **"Разморозка" и продолжение обучения:** веса предобученной модели используются как отправная точка. Все слои модели "размораживаются" (становятся обучаемыми), и процесс обучения продолжается на новом датасете. Модель продолжает учиться, адаптируя все свои внутренние представления к особенностям нового набора данных.
4.  **Тонкая настройка параметров:** скорость обучения, размер батча и другие гиперпараметры могут быть скорректированы для достижения наилучших результатов на новом датасете. Обычно для fine-tuning используются меньшие скорости обучения по сравнению с начальным обучением, чтобы не "забыть" то, что модель уже выучила.
5.  **Оценка и использование:** после дообучения модель оценивается на тестовых данных, и, если результаты удовлетворительны, она готова к использованию для генерации текста, суммаризации или других задач, на которые она была настроена.

**Преимущества Full Fine-tuning:**

* **Потенциально лучшая производительность:** поскольку все параметры модели адаптируются, full fine-tuning часто может достичь наивысшей производительности на целевой задаче, особенно если новый датасет достаточно велик и существенно отличается от исходных данных.
* **Гибкость:** модель полностью перестраивается под новый домен.

**Недостатки Full Fine-tuning:**

* **Высокие вычислительные затраты:** требует значительных вычислительных ресурсов (GPU/TPU) и времени, поскольку обновляются миллионы или миллиарды параметров.
* **Большие требования к памяти:** модель в памяти остается того же размера, что и исходная, так как все веса сохраняются.
* **Риск "катастрофического забывания" (catastrophic forgetting):** если новый датасет слишком мал или слишком сильно отличается, модель может "забыть" общие языковые знания, полученные на этапе предварительного обучения.

In [ ]:
"""
Данный программный код представляет собой демонстрационный туториал по дообучению (fine-tuning) модели GPT-2
на русскоязычном наборе данных Wikibooks. Он охватывает все этапы: от подготовки данных до тренировки
модели и генерации текста. Основная цель — показать процесс адаптации предобученной языковой модели
к специфическому корпусу текстов для улучшения её способности генерировать связный и релевантный текст
на русском языке.
"""

# --- Импорты стандартных библиотек ---
import sqlite3
from typing import List, Any

# --- Импорты сторонних библиотек ---
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    TextDataset,
    AutoConfig,
)

In [ ]:
# --- Константы ---
# Путь к базе данных Wikibooks
WIKIBOOKS_DB_PATH: str = '/kaggle/input/wikibooks-dataset/wikibooks.sqlite'
# Количество строк для загрузки из базы данных
DB_LOAD_LIMIT: int = 3300
# Размер тестовой выборки (20%)
TEST_SIZE: float = 0.2
# Состояние для воспроизводимости разделения данных
RANDOM_STATE: int = 0
# Имя файла для тренировочных данных
TRAIN_FILE_NAME: str = "train.txt"
# Имя файла для валидационных данных
VALID_FILE_NAME: str = "valid.txt"
# Устройство для обучения модели (GPU, если доступно, иначе CPU)
DEVICE: str = "cuda"
# Имя или путь к предобученной модели GPT-2 на русском языке
MODEL_NAME_OR_PATH: str = 'ai-forever/rugpt3small_based_on_gpt2'
# Размер блока для обработки текста токенизатором
BLOCK_SIZE: int = 64
# Директория для сохранения дообученной модели
OUTPUT_DIR: str = "./finetuned_model"
# Количество эпох для обучения
NUM_TRAIN_EPOCHS: int = 10
# Количество шагов накопления градиента
GRADIENT_ACCUMULATION_STEPS: int = 2
# Использование 16-битной точности для ускорения обучения
FP16: bool = True
# Размер батча для обучения на одном устройстве
PER_DEVICE_TRAIN_BATCH_SIZE: int = 64
# Начальная скорость обучения
LEARNING_RATE: float = 0.0002
# Оптимизатор
OPTIMIZER: str = 'adafactor'
# Тип планировщика скорости обучения
LR_SCHEDULER_TYPE: str = 'cosine'
# Шаги сохранения модели
SAVE_STEPS: int = 1000
# Зерно для воспроизводимости результатов
SEED: int = 42
# Путь к чекпоинту для загрузки модели после дообучения
CHECKPOINT_PATH: str = "./finetuned_model/checkpoint-5000"

### 1. Загружаем датасет

In [ ]:
# --- Загрузка и подготовка данных ---
conn: sqlite3.Connection = sqlite3.connect(WIKIBOOKS_DB_PATH)

# Загрузка данных из таблицы 'ru' с ограничением по количеству строк
df: pd.DataFrame = pd.read_sql_query(f"SELECT * FROM ru LIMIT {DB_LOAD_LIMIT}", conn)

In [3]:
df.head()

,title,url,abstract,body_text,body_html
0,Викиучебник: Техника и технология средств масс...,https://ru.wikibooks.org/wiki/%D0%A2%D0%B5%D1%...,* [станция|Рабочая станция];,Рабочая станция;\nСервер;\nПерсональный компью...,"<div class=""mw-parser-output""><ul><li><a href=..."
1,Викиучебник: АОН/Пилотское свидетельство,https://ru.wikibooks.org/wiki/%D0%90%D0%9E%D0%...,Гражданское пилотское свидетельство - разрешен...,В Википедии имеется статья по теме «Свидетельс...,"<div class=""mw-parser-output""><div class=""info..."
2,Викиучебник: Книга программиста/Структуры данн...,https://ru.wikibooks.org/wiki/%D0%9A%D0%BD%D0%...,К оглавлению,"К оглавлению\nВсе программы, код которых вылож...","<div class=""mw-parser-output""><p><a href=""/wik..."
3,Викиучебник: Тесты НМО/Гигиенические основы и ...,https://ru.wikibooks.org/wiki/%D0%A2%D0%B5%D1%...,Гигиенические основы и медицинский контроль за...,Гигиенические основы и медицинский контроль за...,"<div class=""mw-parser-output""><p><b>Гигиеничес..."
4,Викиучебник: Коктейли/Пенная фея,https://ru.wikibooks.org/wiki/%D0%9A%D0%BE%D0%...,Пенная фея,Пенная фея\n\nДжин Old Tom — 60 г\nАбсент — 15...,"<div class=""mw-parser-output""><p><b>Пенная фея..."


In [ ]:
# Фильтрация пустых текстовых полей
df = df[df['body_text'] != '']

### 2. Train test split

In [ ]:
# Разделение данных на тренировочную и тестовую выборки
train_texts: pd.Series
test_texts:  pd.Series

train_texts, test_texts = train_test_split(df['body_text'],
                                           test_size=TEST_SIZE,
                                           random_state=RANDOM_STATE)

In [6]:
train_texts.shape, test_texts.shape

((2635,), (659,))

### Сохраняем тексты в файлы

In [7]:
with open("train.txt", "w") as file:
    file.write("\n".join(train_texts.tolist()))

with open("valid.txt", "w") as file:
    file.write("\n".join(test_texts.tolist()))

### 3. Запускаем дообучение

In [ ]:
# --- Инициализация модели и токенизатора ---
# Инициализация токенизатора GPT-2
tokenizer: GPT2Tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME_OR_PATH)

# Загрузка предобученной модели GPT-2
model: GPT2LMHeadModel = GPT2LMHeadModel.from_pretrained(MODEL_NAME_OR_PATH).to(DEVICE)

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
# Создание объекта TextDataset для тренировочных данных
# Этот объект подготавливает данные для обучения, используя токенизатор
# и заданный размер блока.
train_dataset: TextDataset = TextDataset(tokenizer=tokenizer,
                                         file_path=f'/kaggle/working/{TRAIN_FILE_NAME}',
                                         block_size=BLOCK_SIZE)

# Создание DataCollator для языкового моделирования
# DataCollator отвечает за динамическое создание батчей данных,
# включая маскирование и другие операции, необходимые для обучения.
data_collator: DataCollatorForLanguageModeling = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# Настройка аргументов для обучения
# Здесь определяются различные параметры обучения, такие как выходная директория,
# количество эпох, размер батча и скорость обучения.
training_args: TrainingArguments = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    fp16=FP16,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    optim=OPTIMIZER,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    save_steps=SAVE_STEPS,
    seed=SEED
)

In [ ]:
# Инициализация объекта Trainer
# Trainer инкапсулирует процесс обучения, позволяя легко управлять
# моделью, аргументами обучения, датасетом и коллатором данных.
trainer: Trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# Запуск процесса обучения
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,2.958200
1000,2.520200
1500,2.274900
2000,2.087500
2500,1.918200
3000,1.768100
3500,1.632600
4000,1.506500
4500,1.431100
5000,1.380700


TrainOutput(global_step=5860, training_loss=1.8585257103825592, metrics={'train_runtime': 6702.7479, 'train_samples_per_second': 111.953, 'train_steps_per_second': 0.874, 'total_flos': 2.4489040453632e+16, 'train_loss': 1.8585257103825592, 'epoch': 9.99})

### 4. Генерируем примеры текста

In [ ]:
# --- Функция для генерации текста ---
# Краткое описания назначения функции
def generate(
    prompt: str,
    do_sample: bool = True,
    num_beams: int = 2,
    temperature: float = 1.5,
    top_p: float = 0.9,
    max_length: int = 75
) -> None:
    """
    Description:
    ---------------
        Генерирует текст, используя дообученную модель GPT-2, на основе заданного
        начального промпта. Поддерживает различные параметры генерации для контроля
        качества и разнообразия вывода.

    Args:
    ---------------
        prompt: Начальный текст (затравка) для генерации.

        do_sample: Если True, будет использоваться сэмплирование для генерации,
                   иначе - жадный поиск или beam search.

        num_beams: Количество лучей для beam search. Если do_sample=False,
                   увеличение num_beams может улучшить качество, но увеличит время.

        temperature: Параметр, влияющий на "креативность" генерации. Более высокие
                     значения делают текст более случайным, низкие - более предсказуемым.

        top_p: Параметр фильтрации вероятностей, при котором выбираются токены,
               сумма вероятностей которых достигает top_p. Помогает избежать
               генерации редких и некорректных токенов.

        max_length: Максимальная длина генерируемого текста, включая промпт.

    Returns:
    ---------------
        None: Выводит сгенерированный текст в консоль.

    Raises:
    ---------------
        Exception: Возникает, если модель или токенизатор не загружены или не
                   доступны для устройства.

    Examples:
    ---------------
        >>> generate("Привет, как дела?", max_length=50)
        'Привет, как дела? Я думаю, что все хорошо.'
    """
    # Кодирование входного промпта в идентификаторы токенов и перемещение на устройство
    input_ids: torch.Tensor = tokenizer.encode(prompt, return_tensors="pt").to(DEVICE)

    # Переключение модели в режим оценки
    # Это отключает дропаут и нормализацию батчей, что важно для инференса.
    model.eval()

    # Отключение вычисления градиентов для ускорения инференса и уменьшения потребления памяти
    with torch.no_grad():
        # Генерация текста с использованием заданных параметров
        out: torch.Tensor = model.generate(input_ids,
                                           do_sample=do_sample,
                                           num_beams=num_beams,
                                           temperature=temperature,
                                           top_p=top_p,
                                           max_length=max_length,
                                           )

    # Декодирование сгенерированных идентификаторов токенов обратно в текст и вывод результата.
    print(list(map(tokenizer.decode, out))[0])

In [ ]:
# --- Загрузка дообученной модели и генерация текста ---
# Загрузка конфигурации модели из сохраненного чекпоинта
# Это необходимо для правильной инициализации модели с архитектурой,
# соответствующей дообученной версии.
config: AutoConfig = AutoConfig.from_pretrained(CHECKPOINT_PATH)

In [ ]:
# Инициализация токенизатора GPT-2
# Токенизатор повторно загружается для гарантии корректной работы.
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME_OR_PATH)

# Загрузка предобученной модели GPT-2 с новой конфигурацией и перемещение на устройство
# Модель инициализируется с конфигурацией из дообученного состояния,
# что позволяет использовать ее для генерации текста.
model = GPT2LMHeadModel(config=config).to(DEVICE)

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

In [ ]:
# Генерация текста с использованием дообученной модели
generate("женщина", max_length=30)

женщина тык✂✂ потерять предательство назначения получимварда 123ike геопол геополгрегрегре отдельными 123 проведено путями полиэти контур 123 123 лезетшней Ставрополь Ставропольota


### Выводы

Опять же, используем тот же датасет, что и раньше. Делим его на обучающую и текстовую выборки. Сохраняем преобразованные тексты в обучающий и валидационный файлы. Используем модель ai-forever/rugpt3small_based_on_gpt2, которая адаптирована под русский язык.

Дообучение проводилось на обучающем наборе данных с использованием настроенных параметров. Потери модели уменьшались с каждым шагом обучения.

После завершения обучения была проверена способность модели к генерации текста. Опять же, из-за малости модели текст по промту выдавался невсегда связанный.

**Основные результаты**

- Эффективность дообучения: снижение потерь в процессе обучения указывает на то, что модель успешно адаптировалась к новым данным.
- Качество генерации текста: генерированные моделью тексты были семантически связны и соответствовали заданным промптам.

**Заключение**

Дообучение модели GPT было успешно реализовано, что позволило улучшить качество генерации текстов на русском языке. Предположительно, при увеличении размера модели могли бы получиться более осмысленные выражения.